# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
from pathlib import Path
from pydantic import BaseModel, Field
from typing import Any
from io import StringIO
from language_models.agent.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agent.react import ReActAgent
from language_models.agent.output_parser import OutputFormat
from language_models.settings import settings
from tools.current_date import current_date
from tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from tools.forecasting import get_forecast, get_regions
from pprint import pprint

In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

Understanding LLMs requires balancing algorithmic reasoning and human thought. Algorithmic reasoning is deterministic, producing consistent outcomes from identical inputs, unlike human thought, which is creative and subjective. LLMs lie between these extremes: they are fluent in natural language but do not truly understand what they say. They can execute algorithms but are limited in proficiency. Practically, executing algorithms involves using tools for desired outcomes. LLMs thus represent a hybrid, processing and generating text while relying on external software for algorithm execution.

LLMs are like new employees who need guidance and tools to perform well. They have potential and language skills but depend on external resources to execute tasks effectively. Providing necessary resources ensures their optimal performance, just as with new hires. Neglecting essential guidance or tools may require adjustments, akin to adapting to the needs of a new employee to ensure success.

Consider both system prompts and task prompts:

- **System prompts:** Set general behavior guidelines, such as the persona the LLM should adopt or how it should handle specific tasks and edge cases. For instance, instruct the LLM to check the date before other tasks or respond in a specific format.

- **Task prompts:** For simple queries, a direct question might suffice. For complex tasks, use structured prompts. For example, when classifying an IT ticket, provide clear details about the ticket, user, and date to ensure accurate handling.

To get the best results from LLMs, it's important to craft clear and effective prompts. Prompt engineering is an iterative process. Start with something simple and add more details later. Things to consider:

1. **Be specific:** Provide detailed information to help the LLM understand your query and give tailored responses.
2. **Ask clear questions:** Ask one question at a time to minimize confusion.
3. **Ask follow-up questions:** Clarify incomplete or unclear initial responses with rephrased queries or additional context.
4. **Use full sentences:** Provide context with clear and concise sentences.
5. **Provide examples:** Use examples to help the LLM understand your requirements and respond appropriately.

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.2,
)

The following cell defines a movie review prompt, calls GPT to analyze the sentiment of the review.

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


Below, we've adjusted the prompt to explicitly request a response indicating whether the sentiment of the movie review is positive or negative. The goal is to receive a concise sentiment analysis without additional text, catering to scenarios where we only need a binary classification of sentiment.

In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


We continue to prompt the model to provide a binary sentiment label for the given movie review. The review is positive, so the expected response should be 1 for positive sentiment. This setup is useful when we want to streamline the process of labeling data for sentiment analysis tasks.

In [6]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = """Wow! This movie was incredible. The acting was superb,
and the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


Now, we present an unrelated question to the language model, asking about the weather in Seattle. However, we still provide the system prompt requesting a binary sentiment label for a movie review. This edge case might result in unexpected or irrelevant responses from the language model, demonstrating the importance of providing relevant prompts for accurate analysis.

In [7]:
system_prompt = """Take the following movie review determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(response)

("I'm sorry, I am an AI language model and I don't have access to real-time "
 'weather information. I suggest checking a reliable weather website or using '
 'a weather app to get the most accurate and up-to-date forecast for Seattle.')


To cover edge cases, we provide clear instructions to the language model on how to handle scenarios where the input does not match the expected format. In this case we ask the LLM to respond with -1.

In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

LLMs use various tools to achieve specific goals, streamline operations, and automate tasks. These tools include:

1. **Data retrieval tools:** Extract information from systems or databases using APIs, SDKs, and real-time metrics.
2. **Communication tools:** Facilitate data exchange with external stakeholders via emails, notifications, or alerts.
3. **Data manipulation tools:** Update or modify data within systems, often requiring approval to manage operational impacts.

Specialized tools also exist to handle tasks LLMs struggle with, like performing calculations or accessing current date and time.

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response) # correct answer: $1,289.98

Total Raw Cost = $1,290.98


In the following code cell, we define a simple calculator function and a Pydantic - you can do it in a different way but for simplicity we will use Pydantic - model Calculator to represent its input arguments. We then create a tool instance calculator_tool using the Tool class from our repository, specifying the calculator function, its name, description, and the Pydantic model for argument validation.

By using Pydantic for JSON model schemas, we ensure that the LLM understands how to use the tools effectively. Additionally, Pydantic allows us to validate the LLM responses easily.

In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations",
    args_schema=Calculator
)

print(calculator_tool)

- Tool Name: Calculator, Tool Description: Use this tool when you want to do calculations, Tool Input: {'expression': {'description': 'A math expression', 'title': 'Expression', 'type': 'string'}}


In the system prompt below, we provide clear instructions to the language model on how to utilize the available tools, particularly the calculator tool, to calculate the result accurately based on the given prompt. This is essential for guiding the language model in effectively accessing and leveraging external tools to perform specific tasks, ensuring accurate and helpful responses to user queries.

In this case, we request JSON output from the language model, specifying that it should include a thought, tool, and tool input in its response. This structured format ensures clarity and consistency in the model's outputs, facilitating easy parsing and interpretation. While JSON output is suitable for this demonstration, in practical applications, text responses parsed via regex are often preferred for their simplicity and cost-effectiveness, as they eliminate the need to output JSON formatting elements such as brackets.

In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools:
{calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "Explain your thought. Consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, we need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


By using the input arguments the LLM provided in its response, we can successfully run software code and show the return value or observation to the LLM.

In [12]:
print(calculator(**response["tool_input"]))

1289.98


We introduce an observation to capture the intermediate steps or observations made during the calculation process. Given the model's inability to recall previous conversations, we introduce a mechanism to track and provide previous steps to the model. This ensures that the model can make informed decisions on when to provide the user with the final answer.

It's important to recognize that LLMs can sometimes produce errors in their output, such as generating incorrect JSON or invalid tool input arguments. To address this, it is crucial to highlight these errors to the LLM in hopes that it can correct them. However, the LLM may not always be able to fix its own mistakes. In such instances, after several iterations of feedback and corrections, we may need to provide a fallback response, such as "None" or empty strings.

This iterative process, combined with structured output formats and tool usage, forms the basis of an LLM-powered AI agent. While it may seem like magic in demonstrations, the underlying mechanism is actually quite straightforward. When using a library, keep in mind that the structured output format is generally managed by the library itself.

![ReAct prompting](../img/react.png)

In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools:
{calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}

Observation: tool result
... (this Thought/Tool/Tool input/Observation can repeat N times)

When you know the answer, you MUST use the following JSON format:
{{
  "thought": "I know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = """Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12

This was your previous work:
Thought: The user wants me to calculate the total raw cost. I will use the Calculator tool.
Tool: Calculator
Tool input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}
Observation: Tool response: 1289.98"""

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I know what to respond",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98"
}


## 3. Chat-based AI Agents

AI agents are mainly used in two ways, with one key application being chat-based interactions. Users converse with LLMs for tasks in customer support, recruitment, production adjustments, sales forecasting, etc. This mirrors the intuitive way humans think about language.

### Earthquake

The following code snippet configures an LLM to specialize in answering earthquake-related questions by simulating the expertise of a United States Geological Survey (USGS) expert. It integrates various tools, including those for earthquake information and current date retrieval, to enhance its responses. Additionally, the LLM's output is formatted according to a specified schema, ensuring clarity and consistency in its responses. The structured output format of the final answer will become more useful later on.

In [14]:
current_date_tool = Tool(
    func=current_date,
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

query_earthquakes_tool = Tool(
    func=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    func=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

In [15]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    tools=[current_date_tool, count_earthquakes_tool, query_earthquakes_tool],
    output_format=OutputFormat.STRING,
    iterations=5,
)

The following question demonstrates how the LLM processes a user's query about the number of earthquakes that occurred on the current day. Using the provided tools, the LLM first retrieves the current date and time. This initial step is crucial for identifying and counting the seismic activities for the day. This process shows the power of [Chain-of-Thought](https://arxiv.org/abs/2201.11903), particularly the [ReAct](https://arxiv.org/abs/2210.03629) prompting method, which enables the AI to tackle multi-step problems. By breaking down the task into smaller, manageable steps — first obtaining the current date, then querying the relevant seismic data — the LLM can deliver correct answers.

In [16]:
response = earthquake_agent.invoke({"question": "How many earthquakes occurred today?"})

01/08/24 20:15:51 INFO Prompt:
How many earthquakes occurred today?
01/08/24 20:16:05 INFO Raw Output:
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time set to the beginning of today and the end time set to the current time.

Tool: Current Date

Tool Input: {}
01/08/24 20:16:05 INFO Thought:
To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time set to the beginning of today and the end time set to the current time.
01/08/24 20:16:05 INFO Tool:
Current Date
01/08/24 20:16:05 INFO Tool Input:
{}
01/08/24 20:16:05 INFO Tool Response:
2024-08-01 20:16:05.565037
01/08/24 20:16:15 INFO Raw Output:
Thought: Now that I have the current date and time, I can use the "Count Earthquakes" 

In [17]:
print(response.final_answer)

There have been 173 earthquakes today.


Now we give the LLM a follow-up question where "Show me 3" refers to the earthquakes that occurred today. By keeping track of the chat history, the LLM can understand the context and continuity of the conversation. This enables the LLM to recognize that "today" is the time frame in question and to provide details about three specific earthquakes that occurred on the current day. This capability to maintain and utilize chat history is essential for  multi-turn interactions.

In [18]:
response = earthquake_agent.invoke({"question": "Show me 3."})

01/08/24 20:16:19 INFO Prompt:
Show me 3.
01/08/24 20:16:25 INFO Raw Output:
Thought: To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will set the start time to the beginning of today and the end time to the current time, and limit the results to 3.

Tool: Query Earthquakes

Tool Input: {"start_time": "2024-08-01T00:00:00", "end_time": "2024-08-01T20:16:05.565037", "limit": 3}
01/08/24 20:16:25 INFO Thought:
To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will set the start time to the beginning of today and the end time to the current time, and limit the results to 3.
01/08/24 20:16:25 INFO Tool:
Query Earthquakes
01/08/24 20:16:25 INFO Tool Input:
{'start_time': '2024-08-01T00:00:00', 'end_time': '2024-08-01T20:16:05.565037', 'limit': 3}
01/08/24 20:16:26 INFO Tool Response:
{'type': 'FeatureCollection', 'metadata': {'generated': 1722536186000, 'url': 'https://earthquake.usgs.gov/f

In [19]:
print(response.final_answer)

Here are the details of 3 earthquakes that occurred today:

1. A magnitude 2.1 earthquake occurred 28 km NNE of Paxson, Alaska. [Details here](https://earthquake.usgs.gov/earthquakes/eventpage/ak0249u7hdvz)

2. A magnitude 4.2 earthquake occurred 57 km WNW of Coquimbo, Chile. [Details here](https://earthquake.usgs.gov/earthquakes/eventpage/us6000nheg)

3. A magnitude 2.5 earthquake occurred 20 km N of Skwentna, Alaska. [Details here](https://earthquake.usgs.gov/earthquakes/eventpage/ak0249u7cv1m)


The LLM can also handle straightforward informational questions that don't require the use of additional tools. In this case, the question about the possibility of MegaQuakes (magnitude 10 or larger) can be answered directly by the language model without invoking any external tools. This shows the LLM's ability to recognize when it can rely on its internal knowledge base to provide an accurate response.

In [20]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

01/08/24 20:16:57 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
01/08/24 20:17:09 INFO Raw Output:
Thought: The user is asking about the possibility of a magnitude 10 or larger earthquake, often referred to as a "MegaQuake". This is a question that can be answered based on scientific knowledge about earthquakes and the limitations of the Richter scale.

Final Answer: Theoretically, there is no upper limit to the magnitude of an earthquake. However, in practical terms, scientists believe that it is unlikely for an earthquake of magnitude 10 or larger to occur. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. The magnitude of an earthquake is a measure of the amount of energy released, and a magnitude 10 earthquake would release 31.6 times more energy than a magnitude 9.5 earthquake. The Earth's crust would likely not be able to accumulate the amount of stress required to produce a magnitude 10 earthquake without breaking first.
01/08

In [21]:
pprint(response.final_answer)

('Theoretically, there is no upper limit to the magnitude of an earthquake. '
 'However, in practical terms, scientists believe that it is unlikely for an '
 'earthquake of magnitude 10 or larger to occur. The largest earthquake ever '
 'recorded was a magnitude 9.5 in Chile in 1960. The magnitude of an '
 'earthquake is a measure of the amount of energy released, and a magnitude 10 '
 'earthquake would release 31.6 times more energy than a magnitude 9.5 '
 "earthquake. The Earth's crust would likely not be able to accumulate the "
 'amount of stress required to produce a magnitude 10 earthquake without '
 'breaking first.')


## 4. Service-based AI Agents

Alternatively, LLMs can be integrated into existing applications as services. For instance, a dashboard button could trigger an LLM to generate sales reports. Unlike traditional APIs, LLMs offer data interpretation and content generation, providing insights and actionable recommendations. This involves encapsulating the AI agent within a function that processes user requests and linking it to an API endpoint. Additionally this allows us to automate workflows based on events.

Almost everything in web or mobile applications involves text in some form. This text can be sent to a language model, which can then use the information to perform tasks such as recommending or comparing selected cars. Like chat-based agents, you can equip LLMs with tools to enhance their functionality.

### Contract Drafting

When it comes to using LLMs to draft contracts, it's important to consider that contracts are often lengthy documents, sometimes spanning over 100 pages. This renders conversation-based applications impractical, as LLMs cannot generate such extensive documents, and users prefer not to interact directly with the LLM and manage its outputs themselves.

Here's a potential approach for an application to draft contracts: Rather than expecting the LLM to generate the entire document at once, we can break it down into manageable sections. Users could provide bullet points for each section, allowing the LLM to formulate the corresponding paragraphs and suggest a title for each section. The application would then concatenate these outputs to create a document. Additionally, in practical scenarios, it's likely necessary to grant the LLM access to specific laws or legal references in some manner.

In [22]:
system_prompt = """You are a corporate lawyer. Take the follow bullet points and generate a draft of a section for a contract. Make it lengthy."""

task_prompt = """Bullet points:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=task_prompt,
    prompt_variables=["section", "section_name"],
    tools=None,
    output_format=OutputFormat.OBJECT,
    object_schema=ContractSection,
    iterations=3,
)

With the structured output format outlined earlier, the language model will provide us with both a title and the content for each section. Using the capabilities of structured output formats, we aren't restricted to requesting solely string outputs as the final answer; instead, we can ask for more complex data structures.

In [23]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = str(response.final_answer["title"]) + "\n\n" + str(response.final_answer["content"])
        sections.append(section)
        contract_drafting_agent.chat.reset()
    return "\n\n".join(sections)

In [24]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

invoicing = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect."""

price_conditions = """PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

amendment = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        invoicing,
        price_conditions,
        amendment,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

01/08/24 20:17:09 INFO Prompt:
Bullet points:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
01/08/24 20:17:20 INFO Raw Output:
Thought: The user wants a contract section that clarifies the use of capitalised terms in the contract and their relation to the General Management Agreement (GMA). The section should explain that the terms, whether used in singular or plural form, have the same meaning as in the GMA. 

Final Answer: 
{
  "title": "Use of Capitalised Terms",
  "content": "This section of the contract pertains to the use of capitalised terms within this Amendment. Any term that is capitalised, irrespective of whether it is used in singular or plural form, shall carry the same meaning as defined in the General Management Agreement (GMA). This provision is to ensure consistency and clarity in the interpretation of the terms used throughout this Amendment and the GMA. It is important to note that the use of singular or plural

In [25]:
pprint(contract)

('Use of Capitalised Terms\n'
 '\n'
 'This section of the contract pertains to the use of capitalised terms within '
 'this Amendment. Any term that is capitalised, irrespective of whether it is '
 'used in singular or plural form, shall carry the same meaning as defined in '
 'the General Management Agreement (GMA). This provision is to ensure '
 'consistency and clarity in the interpretation of the terms used throughout '
 'this Amendment and the GMA. It is important to note that the use of singular '
 'or plural forms does not alter the meaning or interpretation of the term. '
 'All parties involved in this agreement are expected to adhere to this '
 'provision for the purpose of maintaining uniformity and avoiding any '
 'potential ambiguity or misunderstanding.\n'
 '\n'
 'Invoicing and Payment Terms\n'
 '\n'
 'In accordance with the terms set forth in this agreement, Clause 12.1(ii) of '
 'the General Master Agreement (GMA) shall be cancelled and substituted as '
 'follows: [*****

### Sentiment Analysis

For straightforward classification tasks, such as analyzing the sentiment of tweets, we can delegate the labeling process to the LLM. What's neat about this approach is that we can store the LLM's reasoning along with the sentiment label, facilitating the development of explainable AI systems.


In this scenario, it's important to note that this isn't a chat-based application where users manually feed tweets to the LLM one by one and record the results themselves. Instead, the process involves automating the sentiment analysis task, where the LLM classifies the sentiment of tweets without direct user intervention.

In [26]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet
36885,0,1565948193,Mon Apr 20 07:54:02 PDT 2009,NO_QUERY,tomokahana,i hate my mom when her being like a crazy angr...
1354284,1,2047020994,Fri Jun 05 13:02:05 PDT 2009,NO_QUERY,TosinB,how does he know me so well??? guess i'm not a...
998429,1,1836263466,Mon May 18 07:40:57 PDT 2009,NO_QUERY,SoftwareTool,Best of Youtube: THE BIRDS &amp; THE BEES: THE...
1043233,1,1957298885,Fri May 29 00:05:15 PDT 2009,NO_QUERY,Marek_Miller,http://blip.pl/s/10390425 i nie s? to quizy #...
1372343,1,2051242808,Fri Jun 05 21:18:05 PDT 2009,NO_QUERY,JMBrammer,@luv2sing94 sweet dreams The graduation was ...
644281,0,2235986936,Fri Jun 19 02:34:23 PDT 2009,NO_QUERY,foreverandaday,My Stp.brothers wife ran off&amp; left him wit...
81029,0,1752558029,Sat May 09 22:15:40 PDT 2009,NO_QUERY,Christyxcore,"@iGerard I'm sorry, but I just lol'd. That suc..."
78498,0,1751387462,Sat May 09 19:19:56 PDT 2009,NO_QUERY,surenemurene,"wants a mustard colour trenchcoat! Ques is, wh..."
781827,0,2323501793,Thu Jun 25 00:47:37 PDT 2009,NO_QUERY,EmilyGreen09,I want a chubby hug from a little girl called ...
1293330,1,2003186799,Tue Jun 02 06:03:13 PDT 2009,NO_QUERY,nicosiaoceania,@likeomgitserika It was my pleasure.


In [27]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=250,
    temperature=0.2,
)

class Tweet(BaseModel):
    reason: str = Field(description="The reason why you chose the sentiment")
    sentiment: int = Field(description="The sentiment of the tweet")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Tweet:\n{tweet}",
    prompt_variables=["tweet"],
    tools=None,
    output_format=OutputFormat.OBJECT,
    object_schema=Tweet,
    iterations=3,
)

In [28]:
def classify_sentiment(row) -> pd.Series:
    response = sentiment_analysis_agent.invoke({'tweet': row.tweet})
    reason = response.final_answer['reason'] or ""
    sentiment = response.final_answer['sentiment'] or 0
    sentiment_analysis_agent.chat.reset()
    return pd.Series([sentiment, reason], index=['prediction', 'reason'])

In [29]:
df_tweets_sampled[['prediction', 'reason']] = df_tweets_sampled.apply(classify_sentiment, axis=1)

01/08/24 20:19:44 INFO Prompt:
Tweet:
i hate my mom when her being like a crazy angry child 
01/08/24 20:19:51 INFO Raw Output:
Thought: The tweet expresses negative emotions such as hate and anger, so the sentiment is negative.

Final Answer: {'reason': 'The tweet expresses negative emotions such as hate and anger.', 'sentiment': 0}
01/08/24 20:19:51 INFO Thought:
The tweet expresses negative emotions such as hate and anger, so the sentiment is negative.
01/08/24 20:19:51 INFO Final Answer:
{'reason': 'The tweet expresses negative emotions such as hate and anger.', 'sentiment': 0}
01/08/24 20:19:51 INFO Prompt:
Tweet:
how does he know me so well??? guess i'm not as &quot;mysterious&quot; as i think i am... 
01/08/24 20:20:01 INFO Raw Output:
Thought: The tweet seems to be expressing surprise and a bit of self-deprecating humor, but it doesn't seem to be negative. The user is surprised that someone knows them well and is joking about not being as "mysterious" as they thought. This does

In [30]:
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet,prediction,reason
36885,0,1565948193,Mon Apr 20 07:54:02 PDT 2009,NO_QUERY,tomokahana,i hate my mom when her being like a crazy angr...,0,The tweet expresses negative emotions such as ...
1354284,1,2047020994,Fri Jun 05 13:02:05 PDT 2009,NO_QUERY,TosinB,how does he know me so well??? guess i'm not a...,1,The tweet is expressing surprise and a bit of ...
998429,1,1836263466,Mon May 18 07:40:57 PDT 2009,NO_QUERY,SoftwareTool,Best of Youtube: THE BIRDS &amp; THE BEES: THE...,1,The tweet does not contain any negative words ...
1043233,1,1957298885,Fri May 29 00:05:15 PDT 2009,NO_QUERY,Marek_Miller,http://blip.pl/s/10390425 i nie s? to quizy #...,0,The tweet is just stating a fact without expre...
1372343,1,2051242808,Fri Jun 05 21:18:05 PDT 2009,NO_QUERY,JMBrammer,@luv2sing94 sweet dreams The graduation was ...,1,"The tweet is expressing positive sentiments, c..."
644281,0,2235986936,Fri Jun 19 02:34:23 PDT 2009,NO_QUERY,foreverandaday,My Stp.brothers wife ran off&amp; left him wit...,0,The tweet describes a negative situation with ...
81029,0,1752558029,Sat May 09 22:15:40 PDT 2009,NO_QUERY,Christyxcore,"@iGerard I'm sorry, but I just lol'd. That suc...",0,The user is acknowledging a negative situation...
78498,0,1751387462,Sat May 09 19:19:56 PDT 2009,NO_QUERY,surenemurene,"wants a mustard colour trenchcoat! Ques is, wh...",-1,The tweet is expressing a desire and asking fo...
781827,0,2323501793,Thu Jun 25 00:47:37 PDT 2009,NO_QUERY,EmilyGreen09,I want a chubby hug from a little girl called ...,1,The tweet expresses a positive sentiment of wa...
1293330,1,2003186799,Tue Jun 02 06:03:13 PDT 2009,NO_QUERY,nicosiaoceania,@likeomgitserika It was my pleasure.,1,The phrase 'It was my pleasure' indicates a po...


### Structuring Unstructured Data

An excellent application of LLMs involves organizing unstructured data, such as text documents. Take, for instance, a collection of job descriptions. While there are various methods to tackle this task, like coding a parser or using optical character recognition, we opt to leverage an LLM for the job. Initially, we define the specific information we wish to extract from the job postings, such as the job title, salary, application instructions, and more. The LLM then looks at each job description and extracts the details for us. Subsequently, we can effortlessly store this dataset as a CSV file for further analysis and use.

In [31]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

With structured output formats, we can now request even more complex data structures. Here, we aim to retrieve strings, integers, and even lists containing strings. Leveraging Pydantic, we can validate the LLM's output. As shown by the dataset below, we can see that the LLM is capable of handling even more complex data structures.

In [32]:
system_prompt = """Take the following job and extract data about the job.

Respond with the job information:
- job title: title of the job.
- job class no: class number.
- job duties: duties of the job.
- open date: when the position was created. Use DD-MM-YYYY.
- salary: the salary ranges.
- deadline: when the application deadline is. Use DD-MM-YYYY.
- application form: online or email or fax.
- where to apply: url or location."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=500,
    temperature=0.2,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Job description:\n{job}",
    prompt_variables=["job"],
    tools=None,
    output_format=OutputFormat.OBJECT,
    object_schema=Job,
    iterations=3,
)

In [33]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
        job_data_agent.chat.reset()
    return pd.DataFrame(data)

In [34]:
df_jobs = extract_jobs(jobs)

01/08/24 20:21:12 INFO Prompt:
Job description:
UTILITY ACCOUNTANT

Class Code:       1511
Open Date:  09-28-18
(Exam Open to All, including Current City Employees)

ANNUAL SALARY

$71,868 to $89,303, $ 73,539 to $ 91,350, $75,439 to $93,730, $78,801 to $97,906, and $85,044 to $105,652

NOTES:

1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer. 
3. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.

DUTIES

A Utility Accountant performs professional accounting, finance, and/or audit-related work. The scope of such work includes the analysis, preparation and maintenance of financial records and reports; treasury and financial activities; and 

In [35]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,Utility Accountant,1511,A Utility Accountant performs professional acc...,28-09-2018,"[$71,868 to $89,303, $73,539 to $91,350, $75,4...",11-10-2018,online,https://www.governmentjobs.com/careers/lacity
1,SENIOR PLUMBING INSPECTOR,4233,A Senior Plumbing Inspector supervises and tra...,18-05-2018,"[$90,410 to $109,306]",31-05-2018,online,https://www.governmentjobs.com/careers/lacity/...
2,Marine Aquarium Program Director,2403,"A Marine Aquarium Program Director plans, dire...",25-08-2017,"[$64,686 to $94,586]",07-09-2017,online,https://www.governmentjobs.com/careers/lacity
3,ROOFER SUPERVISOR,3478,"A Roofer Supervisor assigns, reviews and evalu...",22-06-2018,"[$88,698 (flat-rated)]",05-07-2018,online,https://www.governmentjobs.com/careers/lacity/...
4,COMMUNICATIONS ELECTRICIAN SUPERVISOR,3689,A Communications Electrician Supervisor assign...,27-01-2017,"[$99,347 (Flat Rated), $115,570 to $122,022]",09-02-2017,online,https://www.governmentjobs.com/careers/lacity/...


## 5. Multi-Agents

When tasks assigned to an LLM become too complex, it is essential to divide the AI agent into multiple components. The common approach is to split the AI agent into multiple agents with specific goals, either sequentially chaining them or linking them in a graph-like structure. This is one way to let multiple LLMs collaborate to solve a specific problem.

### Comparing Unstructured Data

When comparing two job descriptions, one approach involves presenting both job postings to an LLM to identify similarities and differences. However, this method may not yield optimal results, as job descriptions often include non-essential information such as "equal employment opportunity" statements. To address this, we can deconstruct the problem by initially tasking an LLM to extract relevant data from each job description. In our case 2 instances as we are comparing 2 jobs. Subsequently, this condensed information can be inputted into a 3rd LLM, tasked with analyzing the extracted data to identify differences and similarities between the two jobs.

The structured format of the input and output fields in the graph enables us to reference data across the entire chain. This flexibility allows us to control exactly what each LLM processes. By tailoring the information each LLM receives, we ensure they have the essential data to solve their specific task, while also preventing unnecessary information. This approach not only reduces costs but also safeguards against misleading inputs that could compromise the quality of the LLM outputs.

However, a concern may arise if an LLM fails to provide the expected output, potentially breaking the entire workflow from a software perspective. As mentioned previously, in the event of a missing output, we use a default response. In this example walkthrough, we output "None" for all fields. Consequently, we pass "None" to all subsequent LLMs, ensuring they can still execute without encountering errors. However, it's important to note that the final output of the chain could also be "None" since the subsequent LLMs won't be able to solve the problem without the necessary input or the LLM could prompt the user to provide more information, telling the user to try again. 
Additionally, you can equip LLMs with tools to enhance their functionality, allowing each model to solve multi-step subproblems before passing the results to the next model in the chain.

| Input |
|------|
| job 1 |
| job 2 |

<br/>

| LLM that extracts job data |
|------|
| `Input` job 1 |
| `Output` job title 1, job duties 1, salary 1 |

<br/>

| LLM that extracts job data |
|------|
| `Input` job 2 |
| `Output` job title 2, job duties 2, salary 2 |

<br/>

| LLM that compares 2 jobs |
|------|
| `Input` job title 1, job duties 1, salary 1, job title 2, job duties 2, salary 2 |
| `Output` differences, similarities |

<br/>

| Output |
|------|
| differences |
| similarities |

In [36]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [37]:
system_prompt = "Take the following job and extract data about the job"

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    temperature=0.2,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [38]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties:
{job1_duties}
Salary:
{salary1}

Job 2:
Job title: {job2_title}
Job duties:
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    temperature=0.2,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [39]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [40]:
response = chain.invoke({"job1": job1, "job2": job2})

17/06/24 08:31:01 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the preparation of designs, plans, specifications

In [41]:
pprint(response.final_answer["similarities"])

('Both jobs are in the field of electrical work and involve dealing with '
 'electrical systems and equipment. They both may require working in various '
 'facilities and buildings.')


In [42]:
pprint(response.final_answer["differences"])

('The Electrical Engineering Associate is more focused on the planning, '
 'design, and technical direction of electrical systems and equipment, and may '
 'also involve code enforcement functions. The Electrical Mechanic, on the '
 'other hand, is more hands-on and involves the installation and maintenance '
 'of electrical circuits and equipment. The Electrical Mechanic may also '
 'require working at heights, near hazardous materials, and in confined '
 'spaces. The salary range for the Electrical Engineering Associate is wider '
 'and generally higher than that of the Electrical Mechanic.')


### Machine Learning Code Generation

Consider using an AI agent to generate machine learning code. Initially, you might think to give the LLM a piece of the dataset to generate the code. But this simple idea might not work well. The LLM could make mistakes, like treating a classification task as a regression because of how the classes are represented numerically.

To fix this, you could have the LLM figure out the problem type first, using insights from the dataset and prompts from the user, before generating any code. While this sounds doable with just one agent, it actually makes things more complicated when it comes to formatting the output. Sometimes, the LLM will output the code along with a description of the problem type and other information, while other times, it will solely respond with the code.

To make things simpler, it's better to split the AI agent into two parts that work together. The 1st LLM looks at the dataset and problem description to figure out what kind of machine learning problem it is, like classification. Then, the 2nd LLM uses this info to generate machine learning code. This split not only makes things easier for the LLMs but also helps humans understand the structure more easily. It also simplifies the process of adjusting the prompts to ensure that the LLM only responds with code.

| Input |
|------|
| problem description |
| dataset size |
| dataset schema |
| dataset snippet |

<br/>

| LLM that determines the machine learning problem |
|------|
| `Input` problem description, dataset size, dataset schema |
| `Output` modeling problem |

<br/>

| LLM that generates machine learning code |
|------|
| `Input` modeling problem, dataset size, dataset schema, dataset snippet |
| `Output` code |

<br/>

| Output |
|------|
| code |

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    temperature=0.2,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [44]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    temperature=0.2,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [45]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [46]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [47]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

17/06/24 08:31:47 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

17/06/24 08:31:53 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, it seems that the user wants to predict the sentiment of tweets, which is a categorical variable. This is a typical example of a classification problem in machine learning, where the

In [48]:
print(response.final_answer["code"])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming df is the DataFrame containing the dataset
X = df['tweet']
y = df['sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Fit and transform the vectorizer on our corpus
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train_tfidf, y_train)

# Predict the labels
y_pred = model.predict(X_test_tfidf)

# Print the classification report
print(classification_report(y_test, y_pred))


### Forecasting

Structured output formats offer another advantage: when chaining blocks together, we're not limited to chaining LLMs only; we can also use any arbitrary function. Since we know the fields the LLM will output, we can simply pass those to run a function that takes these arguments. In this example, we let an LLM extract the region and use the output to run a function for time series forecasting. In practice, it is also common to have LLMs integrate with forecasting models via tools.

| Input |
|------|
| question |

<br/>

| LLM that determines the region |
|------|
| `Input` question |
| `Output` region |

<br/>

| Code block that does the forecasting |
|------|
| `Input` region |
| `Output` forecast |

<br/>

| Output |
|------|
| forecast |

In [50]:
class Forecast(BaseModel):
    region: str = Field(description="A valid region that the model can perform forecasts for.")

def forecast_earthquakes(region: str) -> dict[str, list]:
    if not region:
        return
    df = get_forecast(region)
    today = pd.Timestamp.now()
    df = df.loc[df.Date > today]
    df = df[["Date", "Magnitude Forecast", "Depth Forecast"]]
    return {"forecast": df.to_dict(orient="records")}

In [51]:
forecasting_tool = Tool(
    func=forecast_earthquakes,
    name='Forecast Earthquakes',
    description='Test forecast model on real-time events.', args_schema=Forecast
)

get_regions_tool = Tool(
    func=get_regions,
    name="Find Regions",
    description="Use this tool to access the available regions that can be used for forecasting."
)

In [52]:
task_prompt = """{question}

Use the Find Regions tool to determine the name of the region used for forecasting."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    temperature=0.2,
)

class Region(BaseModel):
    region: str = Field(description="The region used for forecasting.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[get_regions_tool],
    output_format=Region,
    iterations=5,
)

In [53]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool],
    chain_variables=["question"],
)

In [54]:
response = forecast_chain.invoke({"question": "Run a forecast for California."})

17/06/24 08:32:18 INFO Prompt:
Run a forecast for California.

Use the Find Regions tool to determine the name of the region used for forecasting.
17/06/24 08:32:22 INFO Raw response:
{
  "thought": "The user wants to run a forecast for California. I need to use the 'Find Regions' tool to find the correct region name for California.",
  "tool": "Find Regions",
  "tool_input": {}
}
17/06/24 08:32:22 INFO Thought:
The user wants to run a forecast for California. I need to use the 'Find Regions' tool to find the correct region name for California.
17/06/24 08:32:22 INFO Tool:
Find Regions
17/06/24 08:32:22 INFO Tool input:
{}
17/06/24 08:32:24 INFO Tool response:
{'Oregon', 'Hawaii', 'Alaska', 'Chile', 'Montana', 'Aleutian Islands', 'Wyoming', 'Japan', 'Nevada', 'Washington', 'Baja California', 'California', 'Russia', 'Mexico', 'Puerto Rico', 'Turkey', 'Italy', 'Utah', 'Greece', 'Oklahoma', 'Papua New Guinea', 'Indonesia', 'Idaho', 'Philippines', 'Tonga'}
17/06/24 08:32:27 INFO Raw respon

In [55]:
print(response.final_answer["Forecast Earthquakes"])

{'forecast': [{'Date': Timestamp('2024-06-18 00:00:00'), 'Magnitude Forecast': 1.8457183943840727, 'Depth Forecast': 6.146342640427221}, {'Date': Timestamp('2024-06-19 00:00:00'), 'Magnitude Forecast': 1.8276135715228885, 'Depth Forecast': 6.294642236446759}, {'Date': Timestamp('2024-06-20 00:00:00'), 'Magnitude Forecast': 1.8549890834476483, 'Depth Forecast': 5.9804500970617305}]}


### LLM-powered Functions

AI agents can collaborate with each other in a chain-like fashion, as demonstrated earlier. Another approach involves embedding an AI agent within a function that receives arguments, turning it into a tool like any other function in code. The true strength of LLMs lies in their ability to use tools. By leveraging this capability, we can construct a network of interconnected functionalities, where LLMs utilize tools that are powered by LLMs.

This understanding allows us to develop AI agents capable of handling diverse tasks. The following example illustrates this concept. While the combination of domains may seem mismatched in this context, we can repurpose the previously created agents and chains into a tool, which we then integrate into an LLM. This enables the LLM to answer multi-domain questions effectively.

In [56]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [57]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."),
    dataset_schema: str = Field(description="The dataset schema or information."),
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [ ]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    temperature=0.2,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

In [59]:
response = almighty_agent.invoke({"prompt": "How many earthquakes happened today?"})

17/06/24 08:32:28 INFO Prompt:
How many earthquakes happened today?
17/06/24 08:32:32 INFO Raw response:
{
  "thought": "The user is asking about the number of earthquakes that happened today. I will use the Earthquake Agent to get this information.",
  "tool": "Earthquake Agent",
  "tool_input": {"question": "How many earthquakes happened today?"}
}
17/06/24 08:32:32 INFO Thought:
The user is asking about the number of earthquakes that happened today. I will use the Earthquake Agent to get this information.
17/06/24 08:32:32 INFO Tool:
Earthquake Agent
17/06/24 08:32:32 INFO Tool input:
{'question': 'How many earthquakes happened today?'}
17/06/24 08:32:32 INFO Prompt:
How many earthquakes happened today?
17/06/24 08:32:36 INFO Raw response:
{
  "thought": "To answer this question, I need to find out the current date first.",
  "tool": "Current Date",
  "tool_input": {}
}
17/06/24 08:32:36 INFO Thought:
To answer this question, I need to find out the current date first.
17/06/24 08:32

In [60]:
print(response.final_answer["content"])

There have been 65 earthquakes today.


In [61]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

17/06/24 08:32:51 INFO Prompt:
Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:-----------------------------------

In [62]:
print(response.final_answer["content"])

Here is the Python code to train a model that predicts the sentiment of a tweet:

```python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Convert the text data into numerical data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
```

Please replace 'df' with your actual DataFrame variable.
